In [ ]:
""" recover full text from url given an ID"""

b"\xef\xbb\xbfThe Project Gutenberg eBook of The United States Bill of Rights\r\n    \r\nThis ebook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this ebook or online\r\nat www.gutenberg.org. If you are not located in the United States,\r\nyou will have to check the laws of the country where you are located\r\nbefore using this eBook.\r\n\r\nTitle: The United States Bill of Rights\r\n\r\nAuthor: United States\r\n\r\nRelease date: December 1, 1972 [eBook #2]\r\n                Most recently updated: April 1, 2015\r\n\r\nLanguage: English\r\n\r\n\r\n\r\n*** START OF THE PROJECT GUTENBERG EBOOK THE UNITED STATES BILL OF RIGHTS ***\r\n\r\n\r\nAll of the original Project Gutenberg Etexts from the\r\n1970's were produced in ALL CAPS, no lower case.  The\r\ncomputers we used then did

In [2]:
import requests

id_text = 2
url_text = f"https://www.gutenberg.org/cache/epub/{id_text}/pg{id_text}.txt"

text = requests.get(url_text)
text

<Response [200]>

In [8]:
text.content

b"\xef\xbb\xbfThe Project Gutenberg eBook of The United States Bill of Rights\r\n    \r\nThis ebook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this ebook or online\r\nat www.gutenberg.org. If you are not located in the United States,\r\nyou will have to check the laws of the country where you are located\r\nbefore using this eBook.\r\n\r\nTitle: The United States Bill of Rights\r\n\r\nAuthor: United States\r\n\r\nRelease date: December 1, 1972 [eBook #2]\r\n                Most recently updated: April 1, 2015\r\n\r\nLanguage: English\r\n\r\n\r\n\r\n*** START OF THE PROJECT GUTENBERG EBOOK THE UNITED STATES BILL OF RIGHTS ***\r\n\r\n\r\nAll of the original Project Gutenberg Etexts from the\r\n1970's were produced in ALL CAPS, no lower case.  The\r\ncomputers we used then did

In [19]:
# load embedding from azure
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY_4")
azure_openai_api_endpoint = os.getenv("AZURE_OPENAI_API_ENDPOINT_4")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME_4")

from langchain.embeddings import AzureOpenAIEmbeddings 
embedding_model = AzureOpenAIEmbeddings(openai_api_key=azure_openai_api_key,
                                    azure_deployment='text-embedding-3-large',
                                    azure_endpoint=azure_openai_api_endpoint,
                                    openai_api_version="2023-05-15",
                                    chunk_size=500
)

/var/folders/dv/gzhyqctn53s9bh23g7tbvl940000gn/T/ipykernel_30696/4084954619.py:11: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureOpenAIEmbeddings``.
  embedding_model = AzureOpenAIEmbeddings(openai_api_key=azure_openai_api_key,


In [20]:
# load model from azure
# instanciate LLM
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(api_key=azure_openai_api_key,
                      azure_deployment=deployment_name,
                      api_version="2023-12-01-preview",
                      azure_endpoint=azure_openai_api_endpoint,
                      temperature=0.9
                      )

/var/folders/dv/gzhyqctn53s9bh23g7tbvl940000gn/T/ipykernel_30696/4044634815.py:5: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(api_key=azure_openai_api_key,


In [98]:
from pydantic import BaseModel, Field
import requests

# loader
# from langchain_community.document_loaders import TextLoader # not needed we load a string
from langchain.docstore.document import Document # process string in Documents


# splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

# embedding
from langchain.vectorstores import Chroma
from langchain_community.vectorstores import FAISS


# compression
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# define prompt
from langchain.prompts import PromptTemplate

# send query
from langchain.chains import RetrievalQA

# Define the input schema
class GetFullText(BaseModel):
    id: int = Field(description="ID of the text to parse in the function")
    query: str = Field(description="thing to search for")
from langchain.agents import tool


@tool(args_schema=GetFullText)
def QA_retrive_from_text(id: int,query: str):
    """ask a question over a text"""


    id_text = id
    url_text = f"https://www.gutenberg.org/cache/epub/{id_text}/pg{id_text}.txt"


    # Make the request
    response = requests.get(url_text)

    if response.status_code == 200:
        full_text = response.content

    else:
        raise Exception(f"text not available {response.status_code}")
    # load text
    # loader = TextLoader([full_text])
    # doc = loader.load()
    doc = Document(full_text)

    # split for generic texts
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(model_name="gpt-4",
                                                                        chunk_size=5000, 
                                                                        chunk_overlap=50)
    splits = text_splitter.split_documents([doc])
    # embedded vector store
    vectordb = FAISS.from_documents(splits, embedding_model)
    ### sauver embedding en local

    # compression
    compressor = LLMChainExtractor.from_llm(llm)
    compression_retriever = ContextualCompressionRetriever(base_compressor=compressor,
                                                            base_retriever=vectordb.as_retriever(search_kwargs={"k": 20})) # to get maximu diverse context
    # QA retreiver
    qa_chain = RetrievalQA.from_chain_type(llm,
                                            retriever=compression_retriever,
                                            return_source_documents=True)

    return qa_chain({"query": query})

# result["result"]

In [96]:
# use as function
QA_retrive_from_text(12,"what is the personage doing in the train and why does he speak to other?")

{'query': 'what is the personage doing in the train and why does he speak to other?',
 'result': "In the train, the Guard is checking tickets. He speaks to Alice because she doesn't have a ticket, and he gets upset about it. The other passengers join in with comments, effectively encouraging Alice to comply.",
 'source_documents': [Document(metadata={}, page_content='“Tickets, please!” said the Guard, putting his head in at the window. In a moment everybody was holding out a ticket: they were about the same size as the people, and quite seemed to fill the carriage.\n\n“Now then! Show your ticket, child!” the Guard went on, looking angrily at Alice. And a great many voices all said together (“like the chorus of a song,” thought Alice), “Don’t keep him waiting, child! Why, his time is worth a thousand pounds a minute!”\n\n“I’m afraid I haven’t got one,” Alice said in a frightened tone: “there wasn’t a ticket-office where I came from.” And again the chorus of voices went on. “There wasn’t

In [112]:
# use as function if decorator "tool" not present
QA_retrive_from_text(12,"what is the personage doing in the train and why does he speak to other?")

AttributeError: 'str' object has no attribute 'parent_run_id'

In [90]:
# debug
vectordb.max_marginal_relevance_search("what information do you have access to?")
vectordb.as_retriever(search_kwargs={"k": 20}).invoke("what information do you have?")
result = qa_chain({"query": "what is the personage doing in the train and why does he speak to other?"})
result['result']

[Document(metadata={}, page_content='Project Gutenberg™ is synonymous with the free distribution of\r\nelectronic works in formats readable by the widest variety of\r\ncomputers including obsolete, old, middle-aged and new computers. It\r\nexists because of the efforts of hundreds of volunteers and donations\r\nfrom people in all walks of life.\r\n\r\nVolunteers and financial support to provide volunteers with the\r\nassistance they need are critical to reaching Project Gutenberg™’s\r\ngoals and ensuring that the Project Gutenberg™ collection will\r\nremain freely available for generations to come. In 2001, the Project\r\nGutenberg Literary Archive Foundation was created to provide a secure\r\nand permanent future for Project Gutenberg™ and future\r\ngenerations. To learn more about the Project Gutenberg Literary\r\nArchive Foundation and how your efforts and donations can help, see\r\nSections 3 and 4 and the Foundation information page at www.gutenberg.org.\r\n\r\nSection 3. Informat

In [106]:
# check "tool" attributes
QA_retrive_from_text.args  #run({666,"is there any refeerence to the devil here?"})
QA_retrive_from_text.description
QA_retrive_from_text.name

{'id': {'description': 'ID of the text to parse in the function',
  'title': 'Id',
  'type': 'integer'},
 'query': {'description': 'thing to search for',
  'title': 'Query',
  'type': 'string'}}

In [107]:
# format to openai function
from langchain.tools.render import format_tool_to_openai_function
format_tool_to_openai_function(QA_retrive_from_text)

/var/folders/dv/gzhyqctn53s9bh23g7tbvl940000gn/T/ipykernel_30696/1029250998.py:3: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  format_tool_to_openai_function(QA_retrive_from_text)


{'name': 'QA_retrive_from_text',
 'description': 'ask a question over a text',
 'parameters': {'properties': {'id': {'description': 'ID of the text to parse in the function',
    'type': 'integer'},
   'query': {'description': 'thing to search for', 'type': 'string'}},
  'required': ['id', 'query'],
  'type': 'object'}}

In [111]:
# use as openai function
QA_retrive_from_text({"id":666,"query":"is there any refeerence to the devil here?"})

RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 86400 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}